<a href="https://colab.research.google.com/github/pranay2310/Face-Emotion-Recognition/blob/main/Face_Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!wget https://www.dropbox.com/s/si11cws2pyho1bp/archive.zip

--2022-05-24 11:25:18--  https://www.dropbox.com/s/si11cws2pyho1bp/archive.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/si11cws2pyho1bp/archive.zip [following]
--2022-05-24 11:25:18--  https://www.dropbox.com/s/raw/si11cws2pyho1bp/archive.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc37ab4d6fb93d24547e1651ffe5.dl.dropboxusercontent.com/cd/0/inline/Bl0dEBAnHvLnAN4kOFetcanL9P3_jF1_nRSY_GVh5Hn90zE1KlcKffPVCwSX-3YgDvXN7mPdrwcswx7atT5TUTCSyXefg4UCCybIRqP90nu6E-M2da3cQWwVnj88bJPVfqLA-OxCCUbFCZefSVEtZ33X0G4LkjzKLH23G1xqmRgm9Q/file# [following]
--2022-05-24 11:25:19--  https://uc37ab4d6fb93d24547e1651ffe5.dl.dropboxusercontent.com/cd/0/inline/Bl0dEBAnHvLnAN4kOFetcanL9P3_jF1_nRSY_GVh5Hn90zE1KlcKffPVCwSX-3YgDvXN7